In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from utils import initialize_q_table, get_best_params
from RLOpt import ObjModel, train

In [2]:
# Load dataset
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

data = (X_train, y_train, X_test, y_test)

# 1) Random forests Classifier

### Definig parameters

In [3]:
# Model parameters
params = {
            'n_estimators': np.arange(100, 400, 10),
            'criterion': ['gini', 'entropy', 'log_loss'],
            'max_features': ['sqrt', 'log2', None],
            'bootstrap': [True, False]
        }

# Environment parameters
env = ObjModel(RandomForestClassifier, params, data)     # environment
# Training parameters
learning_rate = 1          # Learning rate
max_steps = int(env.observation_space_n / 2)            # Max steps per episode
gamma = 0.9                # Discounting rate

epsilon = 1.0
decay_rate = 0.005         # Exponential decay rate for exploration prob
state_space = env.observation_space_n
action_space = env.action_space_n

### Learning Q-Table

In [4]:
### Learning Q-Table
Qtable_init_s = initialize_q_table(state_space, action_space)
Qtable_s, episod_rewards_s, best_a = train(epsilon, decay_rate, env, max_steps, Qtable_init_s, gamma, learning_rate)

100%|██████████| 270/270 [00:40<00:00,  6.72it/s]


In [5]:
best_params = get_best_params(Qtable_s, env) + [[env.get_params_by_state(best_a), best_a]]

### Evaluate results

In [7]:
states = []
vals = []
for i in best_params:
    states.append(i[1])
    vals.append(env.step(i[1])[1])
    print('State:', i[1], f'Accuracy: {env.step(i[1])[1]}')


State: 516 Accuracy: 0.98
State: 511 Accuracy: 0.98
State: 381 Accuracy: 0.98


In [8]:
print('best acc:', np.max(vals))
print('best params:', env.get_params_by_state(states[np.argmax(vals)]))

best acc: 0.98
best params: {'n_estimators': 380, 'criterion': 'log_loss', 'max_features': 'sqrt', 'bootstrap': True}


# 2) SGD Classifier

In [18]:
# Model parameters
params = {
    'penalty': ['l2', 'l1', 'elasticnet'],
    'loss': ['log_loss', 'hinge', 'modified_huber', 'squared_hinge', 'perceptron'],
    'random_state': [0],
    'alpha': np.arange(0.0001, 0.01, 0.001)
}

# Environment parameters
env = ObjModel(SGDClassifier, params, data)  # environment
# Training parameters
learning_rate = 1  # Learning rate
max_steps = int(env.observation_space_n / 2)  # Max steps per episode
gamma = 0.9  # Discounting rate

epsilon = 1.0
decay_rate = 0.005  # Exponential decay rate for exploration prob
state_space = env.observation_space_n
action_space = env.action_space_n
### Learning Q-Table

### Learning Q-Table

In [19]:
Qtable_init_s = initialize_q_table(state_space, action_space)
Qtable_s, episod_rewards_s, best_a = train(epsilon, decay_rate, env, max_steps, Qtable_init_s, gamma, learning_rate)

100%|██████████| 75/75 [00:00<00:00, 792.97it/s]


In [20]:
best_params = get_best_params(Qtable_s, env) + [[env.get_params_by_state(best_a), best_a]]

### Evaluate results

In [23]:
states = []
vals = []
for i in best_params:
    states.append(i[1])
    vals.append(env.step(i[1])[1])
    print('State:', i[1], f'Accuracy: {env.step(i[1])[1]}')

State: 53 Accuracy: 0.92
State: 53 Accuracy: 0.92
State: 9 Accuracy: 1.0


In [24]:
print('best acc:', np.max(vals))
print('best params:', env.get_params_by_state(states[np.argmax(vals)]))

best acc: 1.0
best params: {'penalty': 'l2', 'loss': 'log_loss', 'random_state': 0, 'alpha': 0.0091}


# 3) Passive Aggressive Classifier

In [25]:
# Model parameters
params = {
    'C': np.arange(0.05, 5, 0.05),
    'loss': ['hinge', 'squared_hinge'],
    'fit_intercept': [True, False],
    'n_jobs': [-1] # The maximum CPU cores count
}

# Environment parameters
env = ObjModel(PassiveAggressiveClassifier, params, data)  # environment
# Training parameters
learning_rate = 1  # Learning rate
max_steps = int(env.observation_space_n / 2)  # Max steps per episode
gamma = 0.9  # Discounting rate

epsilon = 1.0
decay_rate = 0.005  # Exponential decay rate for exploration prob
state_space = env.observation_space_n
action_space = env.action_space_n
### Learning Q-Table

### Learning Q-Table

In [26]:
Qtable_init_s = initialize_q_table(state_space, action_space)
Qtable_s, episod_rewards_s, best_a = train(epsilon, decay_rate, env, max_steps, Qtable_init_s, gamma, learning_rate)

100%|██████████| 198/198 [00:01<00:00, 150.78it/s]


In [27]:
best_params = get_best_params(Qtable_s, env) + [[env.get_params_by_state(best_a), best_a]]

### Evaluate results

In [28]:
states = []
vals = []
for i in best_params:
    states.append(i[1])
    vals.append(env.step(i[1])[1])
    print('State:', i[1], f'Accuracy: {env.step(i[1])[1]}')

State: 86 Accuracy: 0.94
State: 77 Accuracy: 0.82
State: 42 Accuracy: 0.88


In [29]:
print('best acc:', np.max(vals))
print('best params:', env.get_params_by_state(states[np.argmax(vals)]))

best acc: 1.0
best params: {'C': 1.1, 'loss': 'squared_hinge', 'fit_intercept': True, 'n_jobs': -1}


# 4) KNeighbors Classifier

In [30]:
# 3) Passive Aggressive Classifier
# Model parameters
params = {
    'n_neighbors': np.arange(5, 100, 15),
    'weights': ['uniform', 'distance'],
    'algorithm' :['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size':  np.arange(10, 100, 10)
}

# Environment parameters
env = ObjModel(KNeighborsClassifier, params, data)  # environment
# Training parameters
learning_rate = 1  # Learning rate
max_steps = int(env.observation_space_n / 2)  # Max steps per episode
gamma = 0.9  # Discounting rate

epsilon = 1.0
decay_rate = 0.005  # Exponential decay rate for exploration prob
state_space = env.observation_space_n
action_space = env.action_space_n

### Learning Q-Table

In [31]:
Qtable_init_s = initialize_q_table(state_space, action_space)
Qtable_s, episod_rewards_s, best_a = train(epsilon, decay_rate, env, max_steps, Qtable_init_s, gamma, learning_rate)

100%|██████████| 252/252 [00:01<00:00, 249.29it/s]


In [32]:
best_params = get_best_params(Qtable_s, env) + [[env.get_params_by_state(best_a), best_a]]

### Evaluate results

In [33]:
states = []
vals = []
for i in best_params:
    states.append(i[1])
    vals.append(env.step(i[1])[1])
    print('State:', i[1], f'Accuracy: {env.step(i[1])[1]}')

State: 228 Accuracy: 0.92
State: 203 Accuracy: 1.0
State: 175 Accuracy: 1.0


In [34]:
print('best acc:', np.max(vals))
print('best params:', env.get_params_by_state(states[np.argmax(vals)]))

best acc: 1.0
best params: {'n_neighbors': 35, 'weights': 'distance', 'algorithm': 'kd_tree', 'leaf_size': 60}
